## Upload_to_s3

In [1]:
import time

BUCKET_NAME = 'textract-mumbai'

def lambda_handler(event, context):
    body = json.loads(event['body'])   
    content = body['Content']
    file_name = body['FileName']
    final_file_name = file_name +"_"+str(int(round(time.time() * 1000)))
    file_type = body['FileType']
    file_content = base64.b64decode(content)
    file_path = 'Raw_Data/Avlight/out_'+final_file_name+"."+file_type
    s3 = boto3.client('s3')
    try:
        s3_response = s3.put_object(Bucket=BUCKET_NAME, Key=file_path, Body=file_content)
    except Exception as e:
        raise IOError(e)
    return {
        'statusCode': 200,
        'headers': {
                    'Access-Control-Allow-Header': '*',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET'
        },
        'body': json.dumps({
                "FileName": final_file_name,
                "FilePath": file_path
                })
    }

## Preprocessing

In [ ]:
import boto3
import os
import sys
import uuid
from urllib.parse import unquote_plus
from PIL import Image
from PIL.Image import core as _imaging
#old import after this
from pdf2image import convert_from_path,convert_from_bytes
from PIL import ImageFilter,ImageFile
import img2pdf
from img2pdf import convert
import image_cleaner as imgc
#import img2pdf
#import parse
import argparse
import io
import os
import boto3
import math
import json
#end of imports

 
OUTPUT_BUCKET = "mate-mumbai-prod"
PREFIX = "Raw_Data/Avlight/"


s3BucketName = ''

 

 

mySession = boto3.session.Session()
awsRegion = mySession.region_name
s3 = boto3.resource('s3')

def img2bytes(image):
    buf = io.BytesIO()
    image.save(buf, format='JPEG')
    byte_im = buf.getvalue()
    return byte_im
    

 

    
    # filename = "CKM_EB_20200825_00003.pdf"
    

 

 


def lambda_handler(event, context):
    # TODO implement
    
    
    file = event['Records'][0]['s3']['object']['key']
    s3BucketName  = event['Records'][0]['s3']['bucket']['name']
    fileName = file.split('/')[-1][:-4]
    print(fileName)
    #print(subprocess.check_output(['pip','install','Pillow']) )
    print('Processing File : {}'.format(file))
    s3_client = boto3.client('s3')
    file = s3_client.get_object(Bucket=s3BucketName,Key=file)
   
    images = convert_from_bytes(file['Body'].read())        
    print("Processing Started")
    print("PDF has {} pages".format(len(images)))
    
    processed_images=[]
    print("Cleaning Images with filters")
    for imgnum,image in enumerate(images):
        cleaned_image = imgc.cleanImage(image)   
        processed_images.append(cleaned_image)
    
    print("Converting images to PDF Bytes")
    pdfout = img2pdf.convert(*list(map(img2bytes, processed_images)))
    
    boto3.resource('s3',region_name='ap-south-1').Object(s3BucketName,'preprocessed/Avlight/'+fileName+'.pdf').put(Body=pdfout)

    ''' show jupyter code try again
    s3object = s3.Object(s3BucketName, 'preprocessed/Avlight/' + fileName+'.pdf'))
    print('Processed : ',file.replace('Raw Data','preprocessed'))
    s3object.put(
        Body=(bytes(json.dumps(pdfout).encode('UTF-8')))
    )
    '''
    print("Processing Completed")
    #print(formArr)
        
    
    
        
   
    
    
        
    #print(awsRegion)
    
    return {
        'statusCode': 200,
        #'body': json.dumps(formArr)
    }
 

## PostProcessing

In [ ]:
import boto3
import os
import sys
import uuid
from urllib.parse import unquote_plus
from PIL import Image
from PIL.Image import core as _imaging
#old import after this
from pdf2image import convert_from_path,convert_from_bytes
from PIL import ImageFilter,ImageFile
import img2pdf
from img2pdf import convert
import image_cleaner as imgc
#import img2pdf
#import parse
import argparse
import io
import os
import boto3
import math
import json
#end of imports

 
OUTPUT_BUCKET = "mate-mumbai-prod"
PREFIX = "Raw_Data/Avlight/"


s3BucketName = ''

 

 

mySession = boto3.session.Session()
awsRegion = mySession.region_name
s3 = boto3.resource('s3')

def img2bytes(image):
    buf = io.BytesIO()
    image.save(buf, format='JPEG')
    byte_im = buf.getvalue()
    return byte_im
    

 

    
    # filename = "CKM_EB_20200825_00003.pdf"
    

 

 


def lambda_handler(event, context):
    # TODO implement
    
    
    file = event['Records'][0]['s3']['object']['key']
    s3BucketName  = event['Records'][0]['s3']['bucket']['name']
    fileName = file.split('/')[-1][:-4]
    print(fileName)
    #print(subprocess.check_output(['pip','install','Pillow']) )
    print('Processing File : {}'.format(file))
    s3_client = boto3.client('s3')
    file = s3_client.get_object(Bucket=s3BucketName,Key=file)
   
    images = convert_from_bytes(file['Body'].read())        
    print("Processing Started")
    print("PDF has {} pages".format(len(images)))
    
    processed_images=[]
    print("Cleaning Images with filters")
    for imgnum,image in enumerate(images):
        cleaned_image = imgc.cleanImage(image)   
        processed_images.append(cleaned_image)
    
    print("Converting images to PDF Bytes")
    pdfout = img2pdf.convert(*list(map(img2bytes, processed_images)))
    
    boto3.resource('s3',region_name='ap-south-1').Object(s3BucketName,'preprocessed/Avlight/'+fileName+'.pdf').put(Body=pdfout)

    ''' show jupyter code try again
    s3object = s3.Object(s3BucketName, 'preprocessed/Avlight/' + fileName+'.pdf'))
    print('Processed : ',file.replace('Raw Data','preprocessed'))
    s3object.put(
        Body=(bytes(json.dumps(pdfout).encode('UTF-8')))
    )
    '''
    print("Processing Completed")
    #print(formArr)
        
    
    
        
   
    
    
        
    #print(awsRegion)
    
    return {
        'statusCode': 200,
        #'body': json.dumps(formArr)
    }
 

## Add to dynamoDB

In [ ]:
import json
import boto3
import datetime
s3_client = boto3.client('s3')
dynamodb = boto3.resource('dynamodb')

def lambda_handler(event, context):
    # print(event)
    bucket = event['Records'][0]['s3']['bucket']['name']
    # print(bucket)
    json_file = event['Records'][0]['s3']['object']['key']
    # print(str(json_file))
    json_object = s3_client.get_object(Bucket=bucket,Key=json_file)
    # print(json_object)
    jsonFile = json_object['Body'].read()
    # print(jsonFile)
    jsonDict = json.loads(jsonFile)
    
    print(jsonDict['confidence'])
    confidence1 = jsonDict['confidence1']
    print(jsonDict['confidence1'])
    
    #logic to take out the name of the folder
    folders = json_file.split("/")
    FileName = folders[3].split(".")[0]
    status = folders[2]
    #end
    
    # logic starts-  for finding the number of parts
    numberOfParts=0
    lines = []
    bigdata = []
    
    lines = (jsonDict['lines'])
    for line in (lines):
        data = {}
        numberOfParts+=1
        data['itemCode'] = line["3_Item Code "]
        data['quantity'] = line["5_Quantity "]
        data['basicValue'] = line["8_Basic Value "]
        data['cgst'] = line["13_CGST [INR] Amount "]
        data['sgst'] = line["15_[INR] Amount "]
        bigdata.append(data)
        
    # logic ends
    
    # logic for calulating the totalGST, InvoiceTotal[INR], BasicValue, InvoiceTotal- starts
    amounts = []
    amounts = jsonDict['Amounts']
    SGSTOutput = amounts['SGSTOutput']
    CGSTOutput = amounts['CGSTOutput']
    TotalGST = CGSTOutput+SGSTOutput
    BasicValue = amounts['BasicValue']
    if('InvoiceTotal[INR]' in amounts): 
        InvoiceTotal = amounts['InvoiceTotal[INR]']
    else:
        InvoiceTotal = "N/A"
    # logic ends
    
    # calulating totalQuantity
    lines = (jsonDict['lines'])
    totalQuantity=0
    for line in lines:
        totalQuantity += int(line["5_Quantity "])
   
    # ends

    
    #creating the json to be uploaded in DynamoDB - start
    res = {}
    x = datetime.datetime.now()
    res['time'] = str(x)
    if('GSTIN/UIN' in jsonDict):
        res['GSTIN/UIN'] = str(jsonDict['GSTIN/UIN'])
    else:
        res['GSTIN/UIN'] = str("N/A")
 
    if('InvoiceNo' in jsonDict):
        res['InvoiceNo'] = str(jsonDict['InvoiceNo'])
    else:
        res['InvoiceNo'] = str("N/A")
        
    '''res['Supplier Name'] = str(jsonDict["Supplier Name"])
    # res['InvoiceNo'] = str(jsonDict["InvoiceNo"])
    res['InvoiceDate'] = str(jsonDict['InvoiceDate'])
    # res['GSTIN/UINN'] = str(jsonDict['GSTIN/UINN'])
    res['NumberOfParts'] = numberOfParts
    res['TotalGST'] = str(TotalGST)
    res['InvoiceTotal[INR]'] = str(InvoiceTotal)
    res['BasicValue'] = str(BasicValue)
    res['TotalQuantity'] = str(totalQuantity)
    '''
    
    def check(num):
        num = float(num)
        if(num<80):
            return -1
        else:
            return 1
    

    
    res['BasicValue'] = {}
    res['BasicValue'] = json.dumps({'0':str(BasicValue) ,'1':jsonDict['confidence']['BasicValue'] ,'2':check(jsonDict['confidence']['BasicValue']) })
    
    res['CGSTOutput'] = {}
    res['CGSTOutput'] = json.dumps({"0": str(CGSTOutput),'1':jsonDict['confidence']['CGSTOutput'] ,'2':check(jsonDict['confidence']['CGSTOutput']) })
    
    res['SGSTOutput'] = {}
    res['SGSTOutput'] = json.dumps({"0": str(SGSTOutput),'1':jsonDict['confidence']['SGSTOutput'] ,'2':check(jsonDict['confidence']['SGSTOutput']) })
        
    res['GSTIN/UIN'] = {}
    res['GSTIN/UIN'] = json.dumps({'0':str(jsonDict['GSTIN/UIN']) ,'1':jsonDict['confidence']['GSTIN/UIN'] ,'2':check(jsonDict['confidence']['GSTIN/UIN']) })
    
    res['TotalGST'] = {}
    res['TotalGST'] = json.dumps({'0':str(TotalGST) ,'1':jsonDict['confidence']['TotalGST'] ,'2':check(jsonDict['confidence']['TotalGST']) })
    
    res['InvoiceDate'] = {}
    res['InvoiceDate'] = json.dumps({'0':str(jsonDict['InvoiceDate']) ,'1':jsonDict['confidence']['Date'] , '2':check(jsonDict['confidence']['Date']) })
    
    res['InvoiceNo'] = {}
    res['InvoiceNo'] = json.dumps({'0':str(jsonDict["InvoiceNo"]) , '1': jsonDict['confidence']['InvoiceNo'], '2': check(jsonDict['confidence']['InvoiceNo'])})
    
    res['Supplier Name'] = {}
    res['Supplier Name'] = json.dumps({'0':str((jsonDict["Supplier Name"])),'1':(jsonDict['confidence']['Supplier Name']),'2':check(jsonDict['confidence']['Supplier Name'])})
    
    res['lineItem'] = {}
    res['lineItem'] = json.dumps({'0':bigdata, '1': str(confidence1), '2': check(confidence1)})
    
    res['NumberOfParts'] = numberOfParts
    res['InvoiceTotal[INR]'] = str(InvoiceTotal)
    res['TotalQuantity'] = str(totalQuantity)
    
    res["Location"] = "s3/"+str(bucket)
    res["FileName"] = str(FileName)
    res["Status"] = str(status)
    res["Location"] = "https://s3.console.aws.amazon.com/s3/buckets/textract-mumbai-production/"+str(json_file)
    
    #json certion - end
    
    # toSave = json.dumps(res)
    # print(toSave)
    
    print(res)
    
    table = dynamodb.Table('TextRactProductionData')
    response = table.put_item(Item=res)
    print(datetime.datetime.now())
    # print(str(jsonDict))
    return {
        'statusCode': 200,
        # 'body': json.dumps('Hello from Lambda!')
    }

## Extract from DynamoDB

In [ ]:
mport json
from decimal import Decimal
import boto3
from boto3.dynamodb.conditions import Key
from botocore.errorfactory import ClientError

class fakefloat(float):
    def __init__(self, value):
        self._value = value
    def __repr__(self):
        return str(self._value)

def defaultencode(o):
    if isinstance(o, Decimal):
        # Subclass float with custom repr?
        return fakefloat(o)
    raise TypeError(repr(o) + " is not JSON serializable")
    
def key_exists(mykey, mybucket):
    s3 = boto3.resource('s3').Bucket(mybucket)
    try:
        s3.Object(mykey).get()
        return True
    except ClientError as ex:
        if ex.response['Error']['Code'] == 'NoSuchKey':
            print('NoSuchKey')
        return False
    pass
  
def lambda_handler(event, context):
    # TODO implement
    body = json.loads(event['body'])   
    file = body['filename']
    filetype = body['filetype']
    
    if file == '':
        return {
                'statusCode': 200,
                'headers': {
                    'Access-Control-Allow-Header': '*',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET'
                },
                'body': json.dumps({
                    'Status' : 'Error',
                    'Message' : 'No File Name'
                })
        }
    
    TABLE_NAME = "TextRactProduction" 

    # Creating the DynamoDB Table Resource
    dynamodb = boto3.resource('dynamodb', region_name="ap-south-1")
    table = dynamodb.Table(TABLE_NAME)
    
    response = table.get_item(
    Key={
        'FileName': file
    }
    )
    
    if 'Item' in response:
        final_json = response['Item']
        final_json['InvoiceTotal'] = final_json['InvoiceTotal[INR]']
        final_json['SupplierName'] = final_json['Supplier Name']
        final_json['GSTIN'] = final_json['GSTIN/UIN']
        
        msg= ''
        for key in final_json:
            if(final_json[key] == "N/A"):
                msg = 'Some value is invalid, need manual processing'
                break
            else:
                msg= 'Successfully uploaded to database'
        final_json['Message']= msg
        return {
            'statusCode': 200,
            'headers': {
                'Access-Control-Allow-Headers': '*',
                'Access-Control-Allow-Origin': '*',
                'Access-Control-Allow-Methods': 'OPTIONS,POST,GET'
            },
            'body': json.dumps(final_json, default=defaultencode)
        }

    else:
        if key_exists('Raw_Data/Avlight/out_'+file+'.'+filetype,'textract-mumbai-production'):
            return {
                'statusCode': 200,
                'headers': {
                    'Access-Control-Allow-Header': '*',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET'
                },
                'body': json.dumps({
                    'Status' : 'In Progress',
                    'Message' : 'File is being processed. Please wait for a while and try again'
                })
            } 
        else:
            return {
                'statusCode': 200,
                'headers': {
                    'Access-Control-Allow-Header': '*',
                    'Access-Control-Allow-Origin': '*',
                    'Access-Control-Allow-Methods': 'OPTIONS,POST,GET'
                },
                'body': json.dumps({
                    'Status' : 'Error',
                    'Message' : 'Could not be uploaded, need manual processing'
                })
            } 

    